In [4]:
import pickle
from os.path import join
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Dataframe
path = 'D:\\hse\\3 course\\course_work\\CourseWork2020\\02. News Classifier\\02. Feature Engineering\\Pickles'
path_df = join(path, 'df.pickle')
with open(path_df, 'rb') as data:
    df = pickle.load(data)

# features_train
path_features_train = join(path, 'features_train.pickle')
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

# labels_train
path_labels_train = join(path, 'labels_train.pickle')
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

# features_test
path_features_test = join(path, 'features_test.pickle')
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

# labels_test
path_labels_test = join(path, 'labels_test.pickle')
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)